# LaneWeedNet‑Lite — Colab demo notebook
Quick smoke‑test for training & inference on 4 equal lanes.

**Repo:** https://github.com/bassyboi/Laneweednet-lite

What this does:
1. Check GPU
2. Clone repo & install requirements
3. (Optional) Mount Google Drive
4. Prepare data (COCO ➜ lane labels)
5. Train a tiny run (epochs=5 by default)
6. Inference on a single image + batch of images


In [ ]:
#@title 1) Check GPU
import torch, platform
print('Python:', platform.python_version())
print('Torch:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('CUDA device:', torch.cuda.get_device_name(0))
else:
    print('⚠️ Using CPU. Training will be slower.')


In [ ]:
#@title 2) Clone repo & install requirements
%cd /content
!rm -rf Laneweednet-lite
!git clone https://github.com/bassyboi/Laneweednet-lite
%cd /content/Laneweednet-lite

# Colab already has torch; avoid downgrades. If requirements.txt pins torch, you can allow it.
!pip install -r requirements.txt --quiet

print('Setup complete.')


In [ ]:
#@title 3) (Optional) Mount Google Drive
use_drive = False  #@param {type:"boolean"}
if use_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Drive mounted at /content/drive')
else:
    print('Skipping Drive mount.')


## 4) Prepare data
You have two paths:

**A. Already have lane label JSONs** `lanes_train.json` and `lanes_val.json` (with 4-lane targets)? Place them in `/content/Laneweednet-lite` or any folder and update paths below.

**B. Start from COCO (boxes) ➜ convert to lane labels:** Provide your COCO train/val JSONs and image folder, then run the converter cell.


In [ ]:
#@title 4B) Convert COCO ➜ lane labels (one-off)
COCO_TRAIN_JSON = '/content/data/train.json'  #@param {type:"string"}
COCO_VAL_JSON   = '/content/data/val.json'    #@param {type:"string"}
IMG_DIR         = '/content/data/images'      #@param {type:"string"}
IMG_SIZE        = 384                         #@param {type:"integer"}
LANES           = 4                           #@param {type:"integer"}
AREA_THRESH     = 4800                        #@param {type:"integer"}
OUT_TRAIN       = '/content/Laneweednet-lite/lanes_train.json'  #@param {type:"string"}
OUT_VAL         = '/content/Laneweednet-lite/lanes_val.json'    #@param {type:"string"}

%cd /content/Laneweednet-lite
print('Converting TRAIN…')
!python lane_labels_from_coco.py \
  --coco "$COCO_TRAIN_JSON" \
  --img_dir "$IMG_DIR" \
  --img_size $IMG_SIZE \
  --lanes $LANES \
  --area_thresh $AREA_THRESH \
  --out "$OUT_TRAIN"

print('Converting VAL…')
!python lane_labels_from_coco.py \
  --coco "$COCO_VAL_JSON" \
  --img_dir "$IMG_DIR" \
  --img_size $IMG_SIZE \
  --lanes $LANES \
  --area_thresh $AREA_THRESH \
  --out "$OUT_VAL"
print('Done.')


In [ ]:
#@title 5) Train — fast sanity run
TRAIN_JSON = '/content/Laneweednet-lite/lanes_train.json'  #@param {type:"string"}
VAL_JSON   = '/content/Laneweednet-lite/lanes_val.json'    #@param {type:"string"}
IMG_DIR    = '/content/data/images'                        #@param {type:"string"}
IMG_SIZE   = 384                                           #@param {type:"integer"}
EPOCHS     = 5                                             #@param {type:"integer"}

%cd /content/Laneweednet-lite
!python train_lanes.py \
  --train "$TRAIN_JSON" \
  --val "$VAL_JSON" \
  --img_dir "$IMG_DIR" \
  --img $IMG_SIZE \
  --epochs $EPOCHS


In [ ]:
#@title 6) Inference — single image
WEIGHTS = 'runs/best.pt'                 #@param {type:"string"}
TEST_IMAGE = '/content/data/test.jpg'    #@param {type:"string"}
IMG_SIZE = 384                           #@param {type:"integer"}
%cd /content/Laneweednet-lite
!python infer_lanes.py \
  --weights "$WEIGHTS" \
  --image "$TEST_IMAGE" \
  --img $IMG_SIZE


In [ ]:
#@title 7) Inference — folder (prints lane probs per image)
WEIGHTS = 'runs/best.pt'                      #@param {type:"string"}
FOLDER  = '/content/data/frames'              #@param {type:"string"}
IMG_SIZE= 384                                 #@param {type:"integer"}
%cd /content/Laneweednet-lite
import glob, os, json, subprocess
imgs = sorted(glob.glob(os.path.join(FOLDER, '*.jpg')) + 
              glob.glob(os.path.join(FOLDER, '*.png')))
print('Found', len(imgs), 'images')
for i, img in enumerate(imgs[:50]):
    print(f'[{i+1}/{len(imgs)}] {os.path.basename(img)}')
    cmd = [
        'python', 'infer_lanes.py',
        '--weights', WEIGHTS,
        '--image', img,
        '--img', str(IMG_SIZE)
    ]
    subprocess.run(cmd, check=False)


### Notes
- If you see `CUDA not available`, your Colab runtime likely lacks a GPU. Switch to *Runtime → Change runtime type → GPU*.
- If lane labels come out skewed (too many 1s or 0s), adjust `AREA_THRESH` in the converter cell.
- For on‑rig triggering, smooth the four probabilities with a small EMA + hysteresis.
